In [21]:
#Import Relevant Libraries
import random
import numpy as np
import cvxpy as cp

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

from qiskit import quantum_info as qi
from toqito.channel_metrics import channel_fidelity
from qiskit.circuit.library.standard_gates import CRYGate
from toqito.state_metrics import fidelity
from toqito.channels import partial_trace as pt
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace

In [2]:
'''
Defines parameters that specify the purification of state to be tested. Create a QuantumCircuit object that
creates the purification on (size) number of qubits.
'''
unitary_param_1 = [3.6394377, 1.98700556, 4.48988233, 4.29453371, 0.99316018, 0.41805292,
                   2.50177143, 3.00526562, 3.68954483, 6.10950746, 5.17815346, 1.62734164,
                   4.37865732, 0.8921942,  6.12110679, 4.63305539]

size = 4
layers = 2
state = QuantumCircuit(size)
for j in range(layers):
    for i in range(size):
        state.rx(unitary_param_1[j*size*2 + i], i)
        state.ry(unitary_param_1[j*2*size + i + size], i)

    for i in range(size-1):
        state.cx(i, i+1)

state.draw('mpl')

In [116]:
'''
Extract the matrix representation of the state \rho_S
'''
psi = np.array(DensityMatrix.from_instruction(state.reverse_bits()))
rho = pt(psi, [3, 4], [2, 2, 2, 2])

[[ 0.03335   +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.46665   +0.j -0.43331786+0.j  0.        +0.j]
 [ 0.        +0.j -0.43331786+0.j  0.46665   +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.03335   +0.j]]


In [26]:
'''
Define matrix representations of required gates.
'''
I = np.array([[1, 0], [0, 1]])
SWAP = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
S12 = np.kron(SWAP, I)
S23 = np.kron(I, SWAP)
S13 = np.array([[1,0,0,0,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,0,1,0], 
                [0,1,0,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,0,0,0,1]])
S12S23 = np.matmul(S12, S23)
S13S23 = np.matmul(S13, S23)

In [28]:
'''
Create list of group elements.
'''
lis = []
lis.append(np.kron(np.kron(I, I), I)) 
lis.append(S23)
lis.append(S13)
lis.append(S13S23)
lis.append(S12)
lis.append(S12S23)

In [117]:
'''
SDP to calculate the Fidelity. Refer to Section 5.
'''
shape = rho.shape

X = cp.Variable(shape, complex=True)
omega = cp.Variable(np.kron(I, S12).shape, complex=True)
G = cp.bmat([[rho, X.H],[X, pt(omega, [3, 4], [2, 2, 2, 2])]])

c = [omega>>0, G>>0, cp.real(cp.trace(omega))==1]

'''
Add the constraint to ensure that sig is G-Symmetric extendible
'''
for item in lis:
    c.append(omega@np.kron(I, item) == np.kron(I, item)@omega)

obj = cp.Maximize(cp.real(cp.trace(X)))
problem = cp.Problem(obj, constraints = c)
rootFid = problem.solve()
print("Root Fidelity is : " + str(rootFid))

WARN: A->p (column pointers) not strictly increasing, column 272 empty
WARN: A->p (column pointers) not strictly increasing, column 391 empty
WARN: A->p (column pointers) not strictly increasing, column 408 empty
WARN: A->p (column pointers) not strictly increasing, column 527 empty
Root Fidelity is : 0.9571878844285074


In [118]:
print("Fidelity is : " + str(rootFid**2))

Fidelity is : 0.9162086460967217
